### Data sources:
    
Census Bureau https://github.com/datamade/census

https://public.opendatasoft.com/ provides latitude and longitude coordinates for all zip codes (saved as Zip_Lat_Lon.json).  

In [16]:
#Import desired tools needed for data collections and analysis

import pandas as pd
import json
import requests
from config import census_key
from census import Census
c = Census(census_key, year=2018)

In [17]:
# Collect Population Data from Census API

#Import Census API data
census_data = c.acs5.get(("NAME", "B01003_001E"), 
                         {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Organize and rename columns
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

census_pd = census_pd[["Zipcode", "Population"]]
                       
census_pd

,Zipcode,Population
0,43964,8642.0
1,28216,51116.0
2,28277,71605.0
3,28278,27286.0
4,28303,29414.0
...,...,...
33115,98279,628.0
33116,98280,417.0
33117,98311,27856.0
33118,98326,657.0


In [18]:
#Read josn file listing all zip codes in KS
#json file was downloaded from https://public.opendatasoft.com/
Zips = pd.read_json("Zip_Lat_Lon.json", orient="list")
print(Zips['geometry'])

0        {'type': 'Point', 'coordinates': [-94.39398, 3...
1        {'type': 'Point', 'coordinates': [-92.16056, 3...
2        {'type': 'Point', 'coordinates': [-94.74357, 4...
3        {'type': 'Point', 'coordinates': [-85.89754, 4...
4        {'type': 'Point', 'coordinates': [-91.98027, 4...
                               ...                        
43186    {'type': 'Point', 'coordinates': [-79.93414, 4...
43187    {'type': 'Point', 'coordinates': [-91.54559, 4...
43188    {'type': 'Point', 'coordinates': [-78.07012, 4...
43189    {'type': 'Point', 'coordinates': [-93.47188, 4...
43190    {'type': 'Point', 'coordinates': [-76.594942, ...
Name: geometry, Length: 43191, dtype: object


In [19]:
Zips.columns

Index(['datasetid', 'recordid', 'fields', 'geometry', 'record_timestamp'], dtype='object')

In [20]:
Zips['fields']

0        {'city': 'Cove', 'zip': '71937', 'dst': 1, 'ge...
1        {'city': 'Edgemont', 'zip': '72044', 'dst': 1,...
2        {'city': 'Sherburn', 'zip': '56171', 'dst': 1,...
3        {'city': 'Lamont', 'zip': '49430', 'dst': 1, '...
4        {'city': 'Richland', 'zip': '52585', 'dst': 1,...
                               ...                        
43186    {'city': 'Pittsburgh', 'zip': '15207', 'dst': ...
43187    {'city': 'Olds', 'zip': '52647', 'dst': 1, 'ge...
43188    {'city': 'Spruce Creek', 'zip': '16683', 'dst'...
43189    {'city': 'Minneapolis', 'zip': '55459', 'dst':...
43190    {'city': 'Pasadena', 'zip': '21123', 'dst': 1,...
Name: fields, Length: 43191, dtype: object

In [21]:
#Select data from "Fields" series and separate dictionaries into data series in new dataframe
#Choose zipcodes for KS and MO only
Zip = pd.concat([Zips.drop(['fields'], axis=1), Zips['fields'].apply(pd.Series)], axis=1)
Zip_Codes = Zip[["city", "state", "zip", "latitude", "longitude"]]
Zip_Codes = Zip_Codes.rename(columns={"zip": "Zipcode"})
Zip_Codes = Zip_Codes.loc[(Zip_Codes["state"] == "MO")|(Zip_Codes["state"] == "KS")]
Zip_Codes["state"].unique()


array(['MO', 'KS'], dtype=object)

In [22]:
#Combine population data with zipcode location data
Population_df = pd.merge(census_pd, Zip_Codes, on="Zipcode")
Population_df.to_csv("Population_by_Zipcode", index=False)
Population_df

,Zipcode,Population,city,state,latitude,longitude
0,63461,5709.0,Palmyra,MO,39.793879,-91.54631
1,63468,2414.0,Shelbina,MO,39.677131,-92.01662
2,63471,644.0,Taylor,MO,39.925941,-91.49636
3,63532,1397.0,Bevier,MO,39.754960,-92.57057
4,63541,346.0,Glenwood,MO,40.513638,-92.60818
...,...,...,...,...,...,...
1713,66010,762.0,Blue Mound,KS,38.102749,-95.01540
1714,67207,28313.0,Wichita,KS,37.669803,-97.23289
1715,67344,931.0,Elk City,KS,37.279439,-95.93171
1716,67859,797.0,Kismet,KS,37.235210,-100.75208
